In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rl-dataset-p2p-server-chuckeet/labeled_network_stats_android 2.csv


# Initial Exploration


* Q(s,a): The current Q-value for a given state s and action a.α (alpha, the learning rate): This controls how much we update our Q-value based on the new information.
* A small alpha means the agent learns slowly.r (the reward): The immediate reward received for taking action a in state s.γ (gamma, the discount factor): This determines the importance of future rewards.
* As you noted, since each chat session is independent, we can set γ = 0. This simplifies our formula greatly!With γ = 0, the update rule becomes:Q(s, a) ← Q(s, a) + α * [r - Q(s, a)]
* In plain English, this means: "The new Q-value is the old Q-value, adjusted slightly by the difference between the actual reward we got (r) and what we expected (Q(s, a))."


In [56]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/rl-dataset-p2p-server-chuckeet/labeled_network_stats_android 2.csv")
print("Successfully loaded data. Shape:", df.shape)
df.head()

Successfully loaded data. Shape: (1189, 14)


,timestamp,isConnected,type,latency(ms),packetLoss(%),download(Mbps),upload(Mbps),bandwidth(Mbps),qos,brand,model,system,version,connection_type
0,2025-07-18T12:17:34.000Z,True,wifi,81,0.0,118.29,10.05,64.17,Excellent,google,sdk_gphone64_arm64,Android,16.0,p2p
1,2025-07-18T12:19:27.138Z,True,wifi,108,0.0,101.39,5.57,53.48,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
2,2025-07-18T12:21:21.061Z,True,wifi,125,0.0,54.59,6.51,30.55,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
3,2025-07-18T12:22:09.429Z,True,wifi,102,0.0,141.95,8.47,75.21,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
4,2025-07-18T12:24:34.642Z,True,wifi,172,0.0,101.39,9.22,55.3,Good,google,sdk_gphone64_arm64,Android,16.0,server


In [57]:
print(set(df['qos']))
print()
print(set(df['connection_type']))
print()


{'Good', 'Excellent', 'google', 'Poor', 'OnePlus', 'Fair', 'Samsung'}

{'p2p', 'server', nan}



In [58]:
set(df['brand'])

{'Apple',
 'CPH2585',
 'OnePlus',
 'SM-A556U',
 'SM-S928U',
 'Samsung',
 'Xiaomi',
 'google',
 'sdk_gphone64_arm64'}

In [59]:
print(df[df['connection_type']=='NaN'])

Empty DataFrame
Columns: [timestamp, isConnected, type, latency(ms), packetLoss(%), download(Mbps), upload(Mbps), bandwidth(Mbps), qos, brand, model, system, version, connection_type]
Index: []


In [60]:
set(df['version'])
df[df['version']=='server']

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,timestamp,isConnected,type,latency(ms),packetLoss(%),download(Mbps),upload(Mbps),bandwidth(Mbps),qos,brand,model,system,version,connection_type
366,2025-07-22T10:16:35.419Z,True,wifi,70,0.0,NaN,NaN,Excellent,Samsung,SM-S928U,Android,16.0,server,NaN
383,2025-07-23T08:02:20.393Z,True,wifi,40,0.0,NaN,NaN,Excellent,Samsung,SM-A556U,Android,16.0,server,NaN
421,2025-07-23T08:05:40.667Z,True,wifi,80,0.0,NaN,NaN,Excellent,google,sdk_gphone64_arm64,Android,16.0,server,NaN
715,2025-07-24T10:01:59.319Z,True,wifi,50,0.0,NaN,NaN,Excellent,OnePlus,CPH2585,Android,15.0,server,NaN


In [61]:
df.head()

,timestamp,isConnected,type,latency(ms),packetLoss(%),download(Mbps),upload(Mbps),bandwidth(Mbps),qos,brand,model,system,version,connection_type
0,2025-07-18T12:17:34.000Z,True,wifi,81,0.0,118.29,10.05,64.17,Excellent,google,sdk_gphone64_arm64,Android,16.0,p2p
1,2025-07-18T12:19:27.138Z,True,wifi,108,0.0,101.39,5.57,53.48,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
2,2025-07-18T12:21:21.061Z,True,wifi,125,0.0,54.59,6.51,30.55,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
3,2025-07-18T12:22:09.429Z,True,wifi,102,0.0,141.95,8.47,75.21,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
4,2025-07-18T12:24:34.642Z,True,wifi,172,0.0,101.39,9.22,55.3,Good,google,sdk_gphone64_arm64,Android,16.0,server


In [62]:
set(df['packetLoss(%)'])

{0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.5,
 1.8,
 1.9,
 2.0,
 2.1,
 2.2,
 2.4,
 2.5,
 2.8,
 3.0,
 3.1,
 3.3,
 3.5,
 3.8,
 4.0,
 4.1,
 4.2,
 4.5,
 5.0,
 100.0}

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189 entries, 0 to 1188
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestamp        1189 non-null   object 
 1   isConnected      1189 non-null   bool   
 2   type             1189 non-null   object 
 3   latency(ms)      1189 non-null   int64  
 4   packetLoss(%)    1189 non-null   float64
 5   download(Mbps)   1170 non-null   float64
 6   upload(Mbps)     1167 non-null   float64
 7   bandwidth(Mbps)  1132 non-null   object 
 8   qos              1189 non-null   object 
 9   brand            1189 non-null   object 
 10  model            1189 non-null   object 
 11  system           1189 non-null   object 
 12  version          1189 non-null   object 
 13  connection_type  1185 non-null   object 
dtypes: bool(1), float64(3), int64(1), object(9)
memory usage: 122.0+ KB


In [64]:
df.head()

,timestamp,isConnected,type,latency(ms),packetLoss(%),download(Mbps),upload(Mbps),bandwidth(Mbps),qos,brand,model,system,version,connection_type
0,2025-07-18T12:17:34.000Z,True,wifi,81,0.0,118.29,10.05,64.17,Excellent,google,sdk_gphone64_arm64,Android,16.0,p2p
1,2025-07-18T12:19:27.138Z,True,wifi,108,0.0,101.39,5.57,53.48,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
2,2025-07-18T12:21:21.061Z,True,wifi,125,0.0,54.59,6.51,30.55,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
3,2025-07-18T12:22:09.429Z,True,wifi,102,0.0,141.95,8.47,75.21,Good,google,sdk_gphone64_arm64,Android,16.0,p2p
4,2025-07-18T12:24:34.642Z,True,wifi,172,0.0,101.39,9.22,55.3,Good,google,sdk_gphone64_arm64,Android,16.0,server



***After my initial exploration i found many discrepencies in data and many irrelevant columns which tend to make the training more complex and tend to over-fit so did initial cleaning and dropping in the next cell***


# Cleaning and Dropping

In [65]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/rl-dataset-p2p-server-chuckeet/labeled_network_stats_android 2.csv")

print("--- Before Cleaning ---")
print("Original Shape:", df.shape)
df.info()

# --- Step 2: Clean Corrupted and Unusable Rows ---

valid_qos = ['Excellent', 'Good', 'Fair', 'Poor']
df = df[df['qos'].isin(valid_qos)]
print(f"\nShape after cleaning 'qos': {df.shape}")

# 2b. Clean the 'connection_type' column
# Our 'action' must be either 'p2p' or 'server'. We drop any rows where this is missing.
df.dropna(subset=['connection_type'], inplace=True)
print(f"Shape after dropping NaN 'connection_type': {df.shape}")

# 2c. Fix the 'bandwidth(Mbps)' column's data type
# The column is an 'object' type, meaning it contains non-numeric text.
# We'll force it to be numeric, and any values that can't be converted will become NaN.
df['bandwidth(Mbps)'] = pd.to_numeric(df['bandwidth(Mbps)'], errors='coerce')
print("'bandwidth(Mbps)' column converted to numeric.")

# --- Step 3: Handle Missing Numerical Data (Imputation) ---

# Instead of deleting rows with missing network stats (and losing valuable data),
# we'll fill the missing spots with the median value of that column.
# The median is more robust to outliers than the mean.
for col in ['download(Mbps)', 'upload(Mbps)', 'bandwidth(Mbps)']:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)
    print(f"NaNs in '{col}' filled with median value: {median_val:.2f}")

# --- Step 4: Drop Irrelevant Columns ---

# As you correctly identified, these columns are not needed for our network-based model.
# 'timestamp' is for logging. 'brand', 'model', 'system', and 'version' are device-specific.
# 'isConnected' appears to have only one value (True), so it's not useful.
columns_to_drop = ['timestamp', 'isConnected', 'brand', 'model', 'system', 'version']
df.drop(columns=columns_to_drop, inplace=True)
print(f"\nDropped irrelevant columns: {columns_to_drop}")

# --- Step 5: Final Verification ---

print("\n\n--- After Cleaning ---")
print("Cleaned Shape:", df.shape)
df.info()

print("\nVerifying unique values in key columns:")
print("QoS levels:", set(df['qos']))
print("Connection types:", set(df['connection_type']))

print("\nFinal dataset head:")
print(df.head())

/tmp/ipykernel_36/1305428350.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)


--- Before Cleaning ---
Original Shape: (1189, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189 entries, 0 to 1188
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestamp        1189 non-null   object 
 1   isConnected      1189 non-null   bool   
 2   type             1189 non-null   object 
 3   latency(ms)      1189 non-null   int64  
 4   packetLoss(%)    1189 non-null   float64
 5   download(Mbps)   1170 non-null   float64
 6   upload(Mbps)     1167 non-null   float64
 7   bandwidth(Mbps)  1132 non-null   object 
 8   qos              1189 non-null   object 
 9   brand            1189 non-null   object 
 10  model            1189 non-null   object 
 11  system           1189 non-null   object 
 12  version          1189 non-null   object 
 13  connection_type  1185 non-null   object 
dtypes: bool(1), float64(3), int64(1), object(9)
memory usage: 122.0+ KB

Shape after cleaning 'qos': (118

# Final features

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1185 entries, 0 to 1188
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   type             1185 non-null   object 
 1   latency(ms)      1185 non-null   int64  
 2   packetLoss(%)    1185 non-null   float64
 3   download(Mbps)   1185 non-null   float64
 4   upload(Mbps)     1185 non-null   float64
 5   bandwidth(Mbps)  1185 non-null   float64
 6   qos              1185 non-null   object 
 7   connection_type  1185 non-null   object 
dtypes: float64(4), int64(1), object(3)
memory usage: 83.3+ KB


# RL FLOW

## Learning 

* Before the agent can learn, we must translate our raw data into the fundamental concepts of Reinforcement Learning: States, Actions, and Rewards. The model doesn't understand "latency" or "p2p"; it only understands numerical indices and values. This step is about creating a clear, unambiguous numerical representation of our problem.

      1. Action Mapping: We'll convert the connection_type column into numerical actions: p2p → 0, server → 1
  
      2. Reward Mapping: We'll convert the qos column into a numerical reward score. This score directly tells the agent how "good" the outcome was.
  
      3. State Discretization: This is the most critical part. A Q-table cannot have an infinite number of states. We must group our continuous numerical features (like latency(ms)) into a finite number of "bins" or "buckets" (e.g., latency_low, latency_medium, latency_high). A state is then defined as a unique combination of these bins.

In [67]:
# config and preprocessing

# action and reward mapping
ACTION_MAP = {'p2p':0,'server':1}
REWARD_MAP = {'Excellent': 1.0, 'Good': 0.75, 'Fair': 0.5, 'Poor': 0.25}

# state discretization bins

LATENCY_BINS = 4      # e.g., low, medium, high, very_high
PACKET_LOSS_BINS = 3  # e.g., none, low, high
BANDWIDTH_BINS = 4    # e.g., slow, medium, fast, very_fast

df['action'] = df['connection_type'].map(ACTION_MAP)

In [68]:
# Map QoS to reward
df['reward'] = df['qos'].map(REWARD_MAP)

# --- Safe binning helper ---
def safe_qcut(series, q, prefix):
    # First pass to see how many bins remain after dropping duplicates
    _, bins = pd.qcut(series, q=q, duplicates='drop', retbins=True)
    n_bins = len(bins) - 1  # number of intervals
    labels = [f'{prefix}_{i}' for i in range(n_bins)]
    return pd.qcut(series, q=q, labels=labels, duplicates='drop')

# 3. Create State Bins
df['latency_bin'] = safe_qcut(df['latency(ms)'], LATENCY_BINS, 'lat')
df['loss_bin'] = safe_qcut(df['packetLoss(%)'], PACKET_LOSS_BINS, 'loss')
df['bw_bin'] = safe_qcut(df['bandwidth(Mbps)'], BANDWIDTH_BINS, 'bw')

# 4. Combine Bins into a Single State String
df['state'] = (
    df['latency_bin'].astype(str) + '_' +
    df['loss_bin'].astype(str) + '_' +
    df['bw_bin'].astype(str) + '_' +
    df['type']  # 'type' is already categorical (wifi/cellular)
)

print("--- Preprocessing Complete ---")
print("Data with State, Action, and Reward:")
print(df[['state', 'action', 'reward']].head())

# Get a list of all possible states and actions for Q-table initialization
states = df['state'].unique()
actions = df['action'].unique()
action_list = list(ACTION_MAP.values())  # Should be [0, 1]

print(f"\nDiscovered {len(states)} unique states.")
print(f"Discovered actions: {action_list}")


--- Preprocessing Complete ---
Data with State, Action, and Reward:
                    state  action  reward
0  lat_1_loss_0_bw_2_wifi       0    1.00
1  lat_1_loss_0_bw_2_wifi       0    0.75
2  lat_2_loss_0_bw_1_wifi       0    0.75
3  lat_1_loss_0_bw_2_wifi       0    0.75
4  lat_2_loss_0_bw_2_wifi       1    0.75

Discovered 32 unique states.
Discovered actions: [0, 1]


# The Q-Table and Training Process

* The Q-table is the brain of our agent. The training process is how we build this brain. We start with a brain that knows nothing (a table of zeros). We then repeatedly show it examples from our historical data. For each example, we tell it: "In this situation (state), this action was taken, and it resulted in this outcome (reward)." The Q-learning formula is the mathematical rule it uses to update its beliefs based on this new information.

## Concept
**Epsilon-Greedy and Epsilon Decay:Exploration (epsilon): If we only ever follow the Q-table, we might get stuck in a rut, always choosing an action that seems "good enough" without ever trying the other option to see if it's better. To avoid this, we sometimes (with probability epsilon) take a random action. This is called exploration.Epsilon Decay: In a production model, we want the agent to explore a lot at the beginning when it knows nothing. As it gets smarter and more confident, we want it to explore less and exploit its knowledge more. We achieve this by gradually reducing epsilon over the training episodes. This leads to faster convergence and a more stable final policy.**


We will initialize a Q-table with zeros. Then, we will create a training loop that iterates through our data multiple times (episodes). Inside the loop, we will apply the Q-learning update rule and decay our epsilon value.

[0, 1]

In [77]:
# --- Step 2b: Q-Table Initialization and Training ---

# --- HYPERPARAMETERS for Training ---
# These are the most important knobs to tune for your model's performance.
config = {
    'num_episodes': 500,        # How many times to loop through the data.
    'learning_rate': 0.1,       # Alpha: How strongly we update Q-values based on new info.
    'start_epsilon': 1.0,       # Start with 100% exploration.
    'end_epsilon': 0.01,        # End with 1% exploration.
}
# Calculate the rate at which epsilon will decay
config['epsilon_decay'] = (config['start_epsilon'] - config['end_epsilon']) / config['num_episodes']


# --- Initialize the Q-Table ---
# The index is our unique states, columns are our actions [0, 1].
# We initialize all values to 0, representing zero knowledge.
q_table = pd.DataFrame(0, index=states, columns=action_list, dtype=np.float64)

print("\n--- Starting Training ---")
epsilon = config['start_epsilon']

# --- The Training Loop ---
for episode in range(config['num_episodes']):
    # In each episode, we iterate through our historical data as our "experience"
    # Shuffling the data in each episode can sometimes help prevent the model from
    # learning order-based patterns that don't actually exist.
    shuffled_df = df.sample(frac=1).reset_index(drop=True)

    for index, row in shuffled_df.iterrows():
        state = row['state']
        action_taken = row['action']
        reward_received = row['reward']

        # --- Q-table Update ---
        # Get the old Q-value for the state-action pair from our table
        old_q_value = q_table.loc[state, action_taken]
        
        # This is the core learning formula for our simplified case (gamma=0)
        # It calculates the "error": the difference between the reward we got and what we expected.
        temporal_difference = reward_received - old_q_value
        
        # The new Q-value is the old value, adjusted by the learning rate times the error.
        new_q_value = old_q_value + config['learning_rate'] * temporal_difference
        
        # Update our Q-table with the new, smarter value
        q_table.loc[state, action_taken] = new_q_value

    # --- Epsilon Decay ---
    # After each full pass through the data, decrease epsilon.
    # We ensure it doesn't go below our defined minimum.
    epsilon = max(config['end_epsilon'], epsilon - config['epsilon_decay'])

    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1}/{config['num_episodes']} completed. Current Epsilon: {epsilon:.4f}")

print("\n--- Training Finished ---")
print("\nFinal Q-Table (sample rows):")
# Let's look at some rows to see what the agent has learned
print(q_table.sample(5))


--- Starting Training ---
Episode 100/500 completed. Current Epsilon: 0.8020
Episode 200/500 completed. Current Epsilon: 0.6040
Episode 300/500 completed. Current Epsilon: 0.4060
Episode 400/500 completed. Current Epsilon: 0.2080
Episode 500/500 completed. Current Epsilon: 0.0100

--- Training Finished ---

Final Q-Table (sample rows):
                                   0         1
lat_0_loss_0_bw_3_wifi      1.000000  1.000000
lat_3_loss_0_bw_3_wifi      0.000000  0.750000
lat_2_loss_0_bw_1_cellular  0.000000  0.639529
lat_2_loss_0_bw_1_wifi      0.848710  0.504291
lat_1_loss_0_bw_1_wifi      0.881195  0.424193


## Prediction function

# Extreme POOR results 

In [82]:
# --- Step 2c: Production Prediction Function ---

# We need the bin edges that were calculated by qcut during training
# to ensure we bin new data in the exact same way.
latency_bins = pd.qcut(df['latency(ms)'], q=LATENCY_BINS, retbins=True, duplicates='drop')[1]
loss_bins = pd.qcut(df['packetLoss(%)'], q=PACKET_LOSS_BINS, retbins=True, duplicates='drop')[1]
bw_bins = pd.qcut(df['bandwidth(Mbps)'], q=BANDWIDTH_BINS, retbins=True, duplicates='drop')[1]

def get_recommendation(latency_ms, packet_loss, bandwidth_mbps, conn_type, q_table_model):
    """
    Recommends a connection type based on live network conditions using the trained Q-table.

    Args:
        latency_ms (int): Current network latency.
        packet_loss (float): Current packet loss percentage.
        bandwidth_mbps (float): Current available bandwidth.
        conn_type (str): 'wifi' or 'cellular'.
        q_table_model (pd.DataFrame): The trained Q-table.

    Returns:
        dict: A dictionary containing the recommended action and the decision details.
    """
    # 1. Discretize the live data using the SAME bins from training
    lat_label = f"lat_{pd.cut([latency_ms], bins=latency_bins, labels=False, include_lowest=True)[0]}"
    loss_label = f"loss_{pd.cut([packet_loss], bins=loss_bins, labels=False, include_lowest=True)[0]}"
    bw_label = f"bw_{pd.cut([bandwidth_mbps], bins=bw_bins, labels=False, include_lowest=True)[0]}"

    # 2. Construct the state string
    state = f"{lat_label}_{loss_label}_{bw_label}_{conn_type}"
    
    # 3. Make a decision
    if state in q_table_model.index:
        # State was seen during training: EXPLOIT
        q_values = q_table_model.loc[state]
        best_action_idx = q_values.idxmax()
        recommendation = 'p2p' if best_action_idx == 0 else 'server'
        decision_info = {
            'state': state,
            'p2p_q_value': round(q_values[0], 4),
            'server_q_value': round(q_values[1], 4),
            'decision': 'model_exploitation'
        }
    else:
        # State is new: Return a safe default
        recommendation = 'server' # Defaulting to server is usually safer
        decision_info = {
            'state': state,
            'p2p_q_value': None,
            'server_q_value': None,
            'decision': 'unseen_state_default'
        }

    return {'recommendation': recommendation, 'details': decision_info}

# --- Example Usage of the Prediction Function ---

# Scenario 1: Excellent WiFi connection
print("\n--- SCENARIO 1: Excellent WiFi ---")
result = get_recommendation(latency_ms=40, packet_loss=0.0, bandwidth_mbps=150, conn_type='wifi', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 2: Unstable Cellular connection
print("\n--- SCENARIO 2: Unstable Cellular ---")
result = get_recommendation(latency_ms=250, packet_loss=2.5, bandwidth_mbps=15, conn_type='cellular', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 3: Very poor WiFi connection (might be an unseen state)
print("\n--- SCENARIO 3: Very Poor WiFi ---")
result = get_recommendation(latency_ms=2000, packet_loss=5.0, bandwidth_mbps=1, conn_type='wifi', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 4:  Cellular connection
print("\n--- SCENARIO 2:  Cellular ---")
result = get_recommendation(latency_ms=20, packet_loss=0.0, bandwidth_mbps=150, conn_type='cellular', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")


--- SCENARIO 1: Excellent WiFi ---
Recommendation: p2p
Details: {'state': 'lat_0_loss_0_bw_3_wifi', 'p2p_q_value': 1.0, 'server_q_value': 1.0, 'decision': 'model_exploitation'}

--- SCENARIO 2: Unstable Cellular ---
Recommendation: server
Details: {'state': 'lat_2_loss_0_bw_0_cellular', 'p2p_q_value': 0.0, 'server_q_value': 0.4213, 'decision': 'model_exploitation'}

--- SCENARIO 3: Very Poor WiFi ---
Recommendation: server
Details: {'state': 'lat_3_loss_0_bw_0_wifi', 'p2p_q_value': 0.0, 'server_q_value': 0.4782, 'decision': 'model_exploitation'}

--- SCENARIO 2:  Cellular ---
Recommendation: server
Details: {'state': 'lat_0_loss_0_bw_3_cellular', 'p2p_q_value': 0.0, 'server_q_value': 0.3747, 'decision': 'model_exploitation'}


# NEW PIPELINE

In [83]:
import pandas as pd
import numpy as np

# --- 1. Configuration: Define Network Profiles ---
# We define different network scenarios to ensure our data covers a wide range of possibilities.
PROFILES = {
    'ideal_wifi': {'type': 'wifi', 'latency': (20, 50), 'loss': (0, 0.1), 'bandwidth': (100, 300), 'count': 200},
    'average_wifi': {'type': 'wifi', 'latency': (50, 150), 'loss': (0.1, 1.0), 'bandwidth': (25, 100), 'count': 400},
    'poor_wifi': {'type': 'wifi', 'latency': (150, 500), 'loss': (1.0, 5.0), 'bandwidth': (1, 25), 'count': 300},
    'excellent_cellular': {'type': 'cellular', 'latency': (40, 80), 'loss': (0, 0.5), 'bandwidth': (50, 200), 'count': 200},
    'average_cellular': {'type': 'cellular', 'latency': (80, 200), 'loss': (0.5, 3.0), 'bandwidth': (10, 50), 'count': 400},
    'poor_cellular': {'type': 'cellular', 'latency': (200, 1000), 'loss': (3.0, 10.0), 'bandwidth': (1, 10), 'count': 300},
    'unstable_high_latency': {'type': 'wifi', 'latency': (500, 4000), 'loss': (0, 15.0), 'bandwidth': (5, 50), 'count': 150},
}

# --- 2. The "Teacher": A Function to Assign Logical QoS (Reward) ---
# This function contains the core logic we want to teach the model.
def get_logical_qos(row):
    """Assigns a QoS rating based on network conditions and connection type."""
    latency, loss, bw, conn_type = row['latency(ms)'], row['packetLoss(%)'], row['bandwidth(Mbps)'], row['connection_type']

    # --- P2P Logic ---
    if conn_type == 'p2p':
        if latency < 60 and loss < 0.2 and bw > 80:
            return 'Excellent'
        if latency < 120 and loss < 1.0 and bw > 40:
            return 'Good'
        if latency < 250 and loss < 4.0 and bw > 15:
            return 'Fair'
        return 'Poor'

    # --- Server Logic ---
    if conn_type == 'server':
        if latency < 150 and loss < 2.0 and bw > 20:
            return 'Excellent' # Server can also be excellent
        if latency < 300 and loss < 5.0 and bw > 10:
            return 'Good' # Server is more resilient to slightly worse conditions
        if latency < 800 and loss < 10.0 and bw > 5:
            return 'Fair' # Server can hold up even in fairly bad conditions
        return 'Poor'

    return 'Fair' # Default case

# --- 3. Data Generation Loop ---
all_data = []

for profile_name, params in PROFILES.items():
    for _ in range(params['count']):
        # For each profile, generate examples for BOTH actions
        for action in ['p2p', 'server']:
            latency = np.random.uniform(params['latency'][0], params['latency'][1])
            loss = np.random.uniform(params['loss'][0], params['loss'][1])
            bandwidth = np.random.uniform(params['bandwidth'][0], params['bandwidth'][1])
            
            # Generate dummy values for other columns for consistency
            download = bandwidth * np.random.uniform(0.9, 1.1)
            upload = bandwidth / np.random.uniform(4.0, 8.0) # Simulate asymmetric connection

            row = {
                'type': params['type'],
                'latency(ms)': int(latency),
                'packetLoss(%)': round(loss, 2),
                'download(Mbps)': round(download, 2),
                'upload(Mbps)': round(upload, 2),
                'bandwidth(Mbps)': round(bandwidth, 2),
                'connection_type': action
            }
            
            # Use our "teacher" function to assign the QoS
            row['qos'] = get_logical_qos(row)
            
            all_data.append(row)

# Create the final DataFrame
synthetic_df = pd.DataFrame(all_data)

# Shuffle the dataset to ensure randomness
synthetic_df = synthetic_df.sample(frac=1).reset_index(drop=True)

print("--- Generated Synthetic Dataset ---")
print("Shape:", synthetic_df.shape)
print("\nSample of generated data:")
print(synthetic_df.head())

print("\nValue counts for connection types (should be balanced):")
print(synthetic_df['connection_type'].value_counts())

# --- 4. Save to CSV ---
# This will save the file in your Kaggle working directory.
# You can then use this new file path in your loading step.
file_path = 'synthetic_network_data.csv'
synthetic_df.to_csv(file_path, index=False)

print(f"\nSuccessfully generated and saved new dataset to '{file_path}'")

--- Generated Synthetic Dataset ---
Shape: (3900, 8)

Sample of generated data:
       type  latency(ms)  packetLoss(%)  download(Mbps)  upload(Mbps)  \
0  cellular          185           1.63           20.92          3.94   
1  cellular           54           0.19           76.40         15.83   
2      wifi           91           0.86           81.36         11.12   
3      wifi          117           0.79           25.63          3.92   
4      wifi          344           4.68           21.36          2.57   

   bandwidth(Mbps) connection_type        qos  
0            22.09             p2p       Fair  
1            77.72          server  Excellent  
2            81.63             p2p       Good  
3            25.08             p2p       Fair  
4            20.51             p2p       Poor  

Value counts for connection types (should be balanced):
connection_type
p2p       1950
server    1950
Name: count, dtype: int64

Successfully generated and saved new dataset to 'synthetic_netw

In [86]:
df_2 = pd.read_csv('/kaggle/working/synthetic_network_data.csv')
df_2.head(20)

,type,latency(ms),packetLoss(%),download(Mbps),upload(Mbps),bandwidth(Mbps),connection_type,qos
0,cellular,185,1.63,20.92,3.94,22.09,p2p,Fair
1,cellular,54,0.19,76.40,15.83,77.72,server,Excellent
2,wifi,91,0.86,81.36,11.12,81.63,p2p,Good
3,wifi,117,0.79,25.63,3.92,25.08,p2p,Fair
4,wifi,344,4.68,21.36,2.57,20.51,p2p,Poor
5,wifi,41,0.03,153.50,30.50,161.09,server,Excellent
6,wifi,119,0.26,54.01,11.75,53.45,p2p,Good
7,cellular,68,0.45,114.90,17.95,112.59,p2p,Good
8,cellular,73,0.19,49.00,7.15,53.58,server,Excellent
9,wifi,96,0.62,90.70,12.62,92.75,p2p,Good


In [87]:
import pandas as pd
import numpy as np

# --- Step 1: Load the NEW High-Quality Dataset ---
# You've already done this, but we'll include it for a complete, runnable script.
df = pd.read_csv('/kaggle/working/synthetic_network_data.csv')

print("--- Successfully loaded new synthetic dataset ---")
print("Shape:", df.shape)


# --- Step 2: Preprocessing and State/Action/Reward Definition ---
# This section defines the structure of our RL problem based on the new data.

# Configuration for Mappings and Bins
ACTION_MAP = {'p2p': 0, 'server': 1}
REWARD_MAP = {'Excellent': 1.0, 'Good': 0.75, 'Fair': 0.5, 'Poor': 0.25}
LATENCY_BINS = 4
PACKET_LOSS_BINS = 3
BANDWIDTH_BINS = 4

# Apply Mappings
df['action'] = df['connection_type'].map(ACTION_MAP)
df['reward'] = df['qos'].map(REWARD_MAP)

# Create State Bins using the new data's distribution
# Using 'duplicates=drop' handles cases where bin edges are not unique, which can
# happen if many data points have the same value (e.g., 0% packet loss).
df['latency_bin'] = pd.qcut(df['latency(ms)'], q=LATENCY_BINS, labels=[f'lat_{i}' for i in range(LATENCY_BINS)], duplicates='drop')
df['loss_bin'] = pd.qcut(df['packetLoss(%)'], q=PACKET_LOSS_BINS, labels=[f'loss_{i}' for i in range(PACKET_LOSS_BINS)], duplicates='drop')
df['bw_bin'] = pd.qcut(df['bandwidth(Mbps)'], q=BANDWIDTH_BINS, labels=[f'bw_{i}' for i in range(BANDWIDTH_BINS)], duplicates='drop')

# Combine bins into the final state representation
df['state'] = (
    df['latency_bin'].astype(str) + '_' +
    df['loss_bin'].astype(str) + '_' +
    df['bw_bin'].astype(str) + '_' +
    df['type']
)

print("\n--- Preprocessing of new data complete ---")
states = df['state'].unique()
action_list = list(ACTION_MAP.values())
print(f"Discovered {len(states)} unique states in the new dataset.")


# --- Step 3: Initialize and Train the New Q-Table ---
# The hyperparameters remain the same, but the training data is now much better.
config = {
    'num_episodes': 500,
    'learning_rate': 0.1,
    'start_epsilon': 1.0,
    'end_epsilon': 0.01,
}
config['epsilon_decay'] = (config['start_epsilon'] - config['end_epsilon']) / config['num_episodes']

# Initialize the Q-table with all zeros
q_table = pd.DataFrame(0, index=states, columns=action_list, dtype=np.float64)

print("\n--- Starting Training on New Dataset---")
epsilon = config['start_epsilon']

for episode in range(config['num_episodes']):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)
    for index, row in shuffled_df.iterrows():
        state = row['state']
        action_taken = row['action']
        reward_received = row['reward']
        
        old_q_value = q_table.loc[state, action_taken]
        temporal_difference = reward_received - old_q_value
        new_q_value = old_q_value + config['learning_rate'] * temporal_difference
        q_table.loc[state, action_taken] = new_q_value
        
    epsilon = max(config['end_epsilon'], epsilon - config['epsilon_decay'])
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1}/{config['num_episodes']} completed. Current Epsilon: {epsilon:.4f}")

print("\n--- Training Finished ---")


# --- Step 4: Production Prediction Function (Recalibrated) ---
# It is CRITICAL to get the new bin edges from the synthetic dataset.
# This ensures that live data is binned in the same way the model was trained.
latency_bins = pd.qcut(df['latency(ms)'], q=LATENCY_BINS, retbins=True, duplicates='drop')[1]
loss_bins = pd.qcut(df['packetLoss(%)'], q=PACKET_LOSS_BINS, retbins=True, duplicates='drop')[1]
bw_bins = pd.qcut(df['bandwidth(Mbps)'], q=BANDWIDTH_BINS, retbins=True, duplicates='drop')[1]

def get_recommendation(latency_ms, packet_loss, bandwidth_mbps, conn_type, q_table_model):
    """Recommends a connection type using the newly trained Q-table."""
    # Discretize live data using the new bin edges
    lat_label = f"lat_{pd.cut([latency_ms], bins=latency_bins, labels=False, include_lowest=True)[0]}"
    # For loss, we need to handle the case where it might be 0
    if packet_loss == 0.0:
        loss_label = 'loss_0'
    else:
        loss_label = f"loss_{pd.cut([packet_loss], bins=loss_bins, labels=False, include_lowest=True)[0]}"

    bw_label = f"bw_{pd.cut([bandwidth_mbps], bins=bw_bins, labels=False, include_lowest=True)[0]}"
    
    state = f"{lat_label}_{loss_label}_{bw_label}_{conn_type}"
    
    if state in q_table_model.index:
        q_values = q_table_model.loc[state]
        best_action_idx = q_values.idxmax()
        recommendation = 'p2p' if best_action_idx == 0 else 'server'
        decision_info = {
            'state': state,
            'p2p_q_value': round(q_values[0], 4),
            'server_q_value': round(q_values[1], 4),
            'decision': 'model_exploitation'
        }
    else:
        recommendation = 'server'
        decision_info = {
            'state': state,
            'p2p_q_value': None,
            'server_q_value': None,
            'decision': 'unseen_state_default'
        }
    return {'recommendation': recommendation, 'details': decision_info}


# --- Step 5: Re-run Scenarios and Analyze the Improvement ---

print("\n\n--- Analysis of New Model ---")
print("Q-Table sample:")
print(q_table.sample(5))

# Scenario 1: Excellent WiFi connection
print("\n--- SCENARIO 1: Excellent WiFi ---")
result = get_recommendation(latency_ms=40, packet_loss=0.0, bandwidth_mbps=150, conn_type='wifi', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 2: Unstable Cellular connection
print("\n--- SCENARIO 2: Unstable Cellular ---")
result = get_recommendation(latency_ms=250, packet_loss=2.5, bandwidth_mbps=15, conn_type='cellular', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 3: Very Poor WiFi connection
print("\n--- SCENARIO 3: Very Poor WiFi ---")
result = get_recommendation(latency_ms=2000, packet_loss=5.0, bandwidth_mbps=1, conn_type='wifi', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 4: The problematic "Good Cellular" case
print("\n--- SCENARIO 4: Good Cellular ---")
result = get_recommendation(latency_ms=60, packet_loss=0.1, bandwidth_mbps=150, conn_type='cellular', q_table_model=q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

--- Successfully loaded new synthetic dataset ---
Shape: (3900, 8)

--- Preprocessing of new data complete ---
Discovered 52 unique states in the new dataset.

--- Starting Training on New Dataset---
Episode 100/500 completed. Current Epsilon: 0.8020
Episode 200/500 completed. Current Epsilon: 0.6040
Episode 300/500 completed. Current Epsilon: 0.4060
Episode 400/500 completed. Current Epsilon: 0.2080
Episode 500/500 completed. Current Epsilon: 0.0100

--- Training Finished ---


--- Analysis of New Model ---
Q-Table sample:
                               0         1
lat_1_loss_0_bw_3_wifi  0.655112  1.000000
lat_3_loss_2_bw_2_wifi  0.250000  0.263601
lat_3_loss_1_bw_1_wifi  0.250000  0.457029
lat_1_loss_1_bw_1_wifi  0.500000  1.000000
lat_3_loss_1_bw_0_wifi  0.250000  0.375673

--- SCENARIO 1: Excellent WiFi ---
Recommendation: server
Details: {'state': 'lat_0_loss_0_bw_3_wifi', 'p2p_q_value': 0.9994, 'server_q_value': 1.0, 'decision': 'model_exploitation'}

--- SCENARIO 2: Unstable Ce

# SHIT RESOLVE


In [88]:
import pandas as pd
import numpy as np

# --- 1. Define Network Profiles (Unchanged) ---
PROFILES = {
    'ideal_wifi': {'type': 'wifi', 'latency': (15, 50), 'loss': (0, 0.1), 'bandwidth': (100, 300), 'count': 200},
    'average_wifi': {'type': 'wifi', 'latency': (50, 150), 'loss': (0.1, 1.0), 'bandwidth': (25, 100), 'count': 400},
    'poor_wifi': {'type': 'wifi', 'latency': (150, 500), 'loss': (1.0, 5.0), 'bandwidth': (1, 25), 'count': 300},
    'excellent_cellular': {'type': 'cellular', 'latency': (40, 80), 'loss': (0, 0.5), 'bandwidth': (50, 200), 'count': 200},
    'average_cellular': {'type': 'cellular', 'latency': (80, 200), 'loss': (0.5, 3.0), 'bandwidth': (10, 50), 'count': 400},
    'poor_cellular': {'type': 'cellular', 'latency': (200, 1000), 'loss': (3.0, 10.0), 'bandwidth': (1, 10), 'count': 300},
    'unstable_high_latency': {'type': 'wifi', 'latency': (500, 4000), 'loss': (0, 15.0), 'bandwidth': (5, 50), 'count': 150},
}

# --- 2. THE NEW "TEACHER": Smarter QoS Logic ---
def get_smarter_qos(row):
    """A more nuanced function to assign QoS based on real-world logic."""
    latency, loss, bw, conn_type = row['latency(ms)'], row['packetLoss(%)'], row['bandwidth(Mbps)'], row['connection_type']
    
    # --- P2P Logic: Prefers low latency, low loss ---
    if conn_type == 'p2p':
        if latency < 60 and loss < 0.2 and bw > 80: return 'Excellent'
        if latency < 120 and loss < 1.0 and bw > 40: return 'Good'
        if latency < 250 and loss < 4.0 and bw > 15: return 'Fair'
        return 'Poor'

    # --- Server Logic: More resilient to latency/loss, but never quite "perfect" in ideal scenarios ---
    if conn_type == 'server':
        # In ideal conditions where P2P would be Excellent, Server is just Good (to represent overhead).
        if latency < 60 and loss < 0.2 and bw > 80: return 'Good' 
        # But it excels in maintaining a connection in worse conditions.
        if latency < 400 and loss < 5.0 and bw > 10: return 'Good'
        if latency < 1000 and loss < 10.0 and bw > 5: return 'Fair'
        return 'Poor'
        
    return 'Fair'

# --- 3. Generate and Save the New Dataset ---
all_data = []
for profile_name, params in PROFILES.items():
    for _ in range(params['count']):
        for action in ['p2p', 'server']:
            row = {
                'type': params['type'],
                'latency(ms)': int(np.random.uniform(params['latency'][0], params['latency'][1])),
                'packetLoss(%)': round(np.random.uniform(params['loss'][0], params['loss'][1]), 2),
                'bandwidth(Mbps)': round(np.random.uniform(params['bandwidth'][0], params['bandwidth'][1]), 2),
                'connection_type': action
            }
            row['qos'] = get_smarter_qos(row)
            all_data.append(row)

smarter_df = pd.DataFrame(all_data)
smarter_df = smarter_df.sample(frac=1).reset_index(drop=True)
file_path = 'smarter_synthetic_data.csv'
smarter_df.to_csv(file_path, index=False)

print(f"--- Generated a new, smarter dataset and saved to '{file_path}' ---")
print("Shape:", smarter_df.shape)

--- Generated a new, smarter dataset and saved to 'smarter_synthetic_data.csv' ---
Shape: (3900, 6)


In [89]:
import pandas as pd
import numpy as np

# --- Step 1: Load the SMARTER High-Quality Dataset ---
df = pd.read_csv('/kaggle/working/smarter_synthetic_data.csv')

print("--- Successfully loaded SMARTER synthetic dataset ---")

# --- Step 2: Preprocessing (Same as before, but on the new data) ---
ACTION_MAP = {'p2p': 0, 'server': 1}
REWARD_MAP = {'Excellent': 1.0, 'Good': 0.75, 'Fair': 0.5, 'Poor': 0.25}
LATENCY_BINS, PACKET_LOSS_BINS, BANDWIDTH_BINS = 4, 3, 4

df['action'] = df['connection_type'].map(ACTION_MAP)
df['reward'] = df['qos'].map(REWARD_MAP)
df['latency_bin'] = pd.qcut(df['latency(ms)'], q=LATENCY_BINS, labels=[f'lat_{i}' for i in range(LATENCY_BINS)], duplicates='drop')
df['loss_bin'] = pd.qcut(df['packetLoss(%)'], q=PACKET_LOSS_BINS, labels=[f'loss_{i}' for i in range(PACKET_LOSS_BINS)], duplicates='drop')
df['bw_bin'] = pd.qcut(df['bandwidth(Mbps)'], q=BANDWIDTH_BINS, labels=[f'bw_{i}' for i in range(BANDWIDTH_BINS)], duplicates='drop')
df['state'] = (df['latency_bin'].astype(str) + '_' + df['loss_bin'].astype(str) + '_' + df['bw_bin'].astype(str) + '_' + df['type'])
states = df['state'].unique()
action_list = list(ACTION_MAP.values())
print(f"Discovered {len(states)} unique states in the new dataset.")

# --- Step 3: Retrain the Q-Table ---
config = {'num_episodes': 500, 'learning_rate': 0.1}
q_table = pd.DataFrame(0, index=states, columns=action_list, dtype=np.float64)

print("\n--- Starting Training on SMARTER Dataset---")
for episode in range(config['num_episodes']):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)
    for index, row in shuffled_df.iterrows():
        state, action, reward = row['state'], row['action'], row['reward']
        old_q_value = q_table.loc[state, action]
        new_q_value = old_q_value + config['learning_rate'] * (reward - old_q_value)
        q_table.loc[state, action] = new_q_value
    if (episode + 1) % 100 == 0: print(f"Episode {episode + 1}/{config['num_episodes']} completed.")
print("\n--- Training Finished ---")


# --- Step 4: THE NEW ROBUST Production Prediction Function ---
latency_bins = pd.qcut(df['latency(ms)'], q=LATENCY_BINS, retbins=True, duplicates='drop')[1]
loss_bins = pd.qcut(df['packetLoss(%)'], q=PACKET_LOSS_BINS, retbins=True, duplicates='drop')[1]
bw_bins = pd.qcut(df['bandwidth(Mbps)'], q=BANDWIDTH_BINS, retbins=True, duplicates='drop')[1]

def get_robust_recommendation(latency_ms, packet_loss, bandwidth_mbps, conn_type, q_table_model):
    """
    Robustly recommends a connection type, handling edge cases and ties.
    """
    # FIX 1: Clip inputs to be within the training data range to prevent NaN errors.
    clipped_latency = np.clip(latency_ms, latency_bins[0], latency_bins[-1])
    clipped_loss = np.clip(packet_loss, loss_bins[0], loss_bins[-1])
    clipped_bw = np.clip(bandwidth_mbps, bw_bins[0], bw_bins[-1])

    lat_label = f"lat_{pd.cut([clipped_latency], bins=latency_bins, labels=False, include_lowest=True)[0]}"
    loss_label = f"loss_{pd.cut([clipped_loss], bins=loss_bins, labels=False, include_lowest=True)[0]}"
    bw_label = f"bw_{pd.cut([clipped_bw], bins=bw_bins, labels=False, include_lowest=True)[0]}"
    
    state = f"{lat_label}_{loss_label}_{bw_label}_{conn_type}"
    
    if state in q_table_model.index:
        q_values = q_table_model.loc[state]
        
        # FIX 2: Implement smart tie-breaking logic.
        # If Q-values are very close (e.g., within 0.05 of each other) and it's an "ideal" state...
        if np.abs(q_values[0] - q_values[1]) < 0.05 and lat_label == 'lat_0' and loss_label == 'loss_0':
            recommendation = 'p2p' # ...prefer P2P.
            decision = 'model_tie-break_preference'
        else:
            best_action_idx = q_values.idxmax()
            recommendation = 'p2p' if best_action_idx == 0 else 'server'
            decision = 'model_exploitation'

        decision_info = {
            'state': state,
            'p2p_q_value': round(q_values[0], 4),
            'server_q_value': round(q_values[1], 4),
            'decision': decision
        }
    else:
        recommendation = 'server' # Safe default for truly new states
        decision_info = {'state': state, 'decision': 'unseen_state_default'}

    return {'recommendation': recommendation, 'details': decision_info}


# --- Step 5: Re-run Scenarios for Final Analysis ---
print("\n\n--- FINAL Analysis of SMARTER Model ---")

# Scenario 1: Excellent WiFi connection
print("\n--- SCENARIO 1: Excellent WiFi ---")
result = get_robust_recommendation(40, 0.0, 150, 'wifi', q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 2: Unstable Cellular connection
print("\n--- SCENARIO 2: Unstable Cellular ---")
result = get_robust_recommendation(250, 2.5, 15, 'cellular', q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 3: Very Poor WiFi connection (The one that crashed)
print("\n--- SCENARIO 3: Very Poor WiFi ---")
result = get_robust_recommendation(2000, 5.0, 1, 'wifi', q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

# Scenario 4: Good Cellular
print("\n--- SCENARIO 4: Good Cellular ---")
result = get_robust_recommendation(60, 0.1, 150, 'cellular', q_table)
print(f"Recommendation: {result['recommendation']}")
print(f"Details: {result['details']}")

--- Successfully loaded SMARTER synthetic dataset ---
Discovered 52 unique states in the new dataset.

--- Starting Training on SMARTER Dataset---
Episode 100/500 completed.
Episode 200/500 completed.
Episode 300/500 completed.
Episode 400/500 completed.
Episode 500/500 completed.

--- Training Finished ---


--- FINAL Analysis of SMARTER Model ---

--- SCENARIO 1: Excellent WiFi ---
Recommendation: p2p
Details: {'state': 'lat_0_loss_0_bw_3_wifi', 'p2p_q_value': 0.9956, 'server_q_value': 0.75, 'decision': 'model_exploitation'}

--- SCENARIO 2: Unstable Cellular ---
Recommendation: server
Details: {'state': 'lat_2_loss_1_bw_1_cellular', 'p2p_q_value': 0.4483, 'server_q_value': 0.75, 'decision': 'model_exploitation'}

--- SCENARIO 3: Very Poor WiFi ---
Recommendation: server
Details: {'state': 'lat_3_loss_2_bw_0_wifi', 'p2p_q_value': 0.25, 'server_q_value': 0.3467, 'decision': 'model_exploitation'}

--- SCENARIO 4: Good Cellular ---
Recommendation: p2p
Details: {'state': 'lat_0_loss_0_bw

In [92]:
q_table

,0,1
lat_3_loss_2_bw_0_cellular,0.250000,0.405581
lat_0_loss_0_bw_3_wifi,0.995620,0.750000
lat_3_loss_2_bw_0_wifi,0.250000,0.346713
lat_0_loss_0_bw_3_cellular,0.793210,0.750000
lat_3_loss_2_bw_1_wifi,0.250000,0.364756
lat_1_loss_1_bw_1_cellular,0.491543,0.750000
lat_2_loss_1_bw_1_cellular,0.448296,0.750000
lat_0_loss_0_bw_2_cellular,0.750000,0.750000
lat_1_loss_2_bw_2_cellular,0.500000,0.750000
lat_2_loss_1_bw_0_wifi,0.250000,0.442915


In [93]:
import pandas as pd
import numpy as np

# Create a list of dictionaries, where each dictionary is a test case.
test_scenarios = [
    # --- Group 1: Ideal & Excellent Conditions ---
    {'desc': "Perfect WiFi", 'params': {'latency_ms': 20, 'packet_loss': 0.0, 'bandwidth_mbps': 250, 'conn_type': 'wifi'}},
    {'desc': "Excellent 5G Cellular", 'params': {'latency_ms': 50, 'packet_loss': 0.1, 'bandwidth_mbps': 180, 'conn_type': 'cellular'}},
    {'desc': "Fast Cable WiFi", 'params': {'latency_ms': 35, 'packet_loss': 0.0, 'bandwidth_mbps': 150, 'conn_type': 'wifi'}},
    
    # --- Group 2: Average / Common Conditions ---
    {'desc': "Average Home WiFi", 'params': {'latency_ms': 70, 'packet_loss': 0.5, 'bandwidth_mbps': 50, 'conn_type': 'wifi'}},
    {'desc': "Good 4G Cellular", 'params': {'latency_ms': 90, 'packet_loss': 1.0, 'bandwidth_mbps': 40, 'conn_type': 'cellular'}},
    {'desc': "Coffee Shop WiFi", 'params': {'latency_ms': 110, 'packet_loss': 1.5, 'bandwidth_mbps': 20, 'conn_type': 'wifi'}},
    {'desc': "City Commute Cellular", 'params': {'latency_ms': 150, 'packet_loss': 3.0, 'bandwidth_mbps': 25, 'conn_type': 'cellular'}},

    # --- Group 3: Poor & Challenging Conditions ---
    {'desc': "Rural Cellular (Weak)", 'params': {'latency_ms': 350, 'packet_loss': 8.0, 'bandwidth_mbps': 5, 'conn_type': 'cellular'}},
    {'desc': "Hotel WiFi (Throttled)", 'params': {'latency_ms': 200, 'packet_loss': 2.0, 'bandwidth_mbps': 3, 'conn_type': 'wifi'}},
    {'desc': "Congested Network", 'params': {'latency_ms': 800, 'packet_loss': 12.0, 'bandwidth_mbps': 10, 'conn_type': 'wifi'}},
    {'desc': "Connection Timeout", 'params': {'latency_ms': 3500, 'packet_loss': 15.0, 'bandwidth_mbps': 2, 'conn_type': 'wifi'}},

    # --- Group 4: High Latency Only ---
    {'desc': "Gaming on Distant Server", 'params': {'latency_ms': 220, 'packet_loss': 0.1, 'bandwidth_mbps': 90, 'conn_type': 'wifi'}},
    {'desc': "Int'l Cellular Roaming", 'params': {'latency_ms': 300, 'packet_loss': 0.5, 'bandwidth_mbps': 50, 'conn_type': 'cellular'}},

    # --- Group 5: High Packet Loss Only ---
    {'desc': "Unstable WiFi Signal", 'params': {'latency_ms': 60, 'packet_loss': 5.0, 'bandwidth_mbps': 60, 'conn_type': 'wifi'}},
    {'desc': "Microwave Interference", 'params': {'latency_ms': 75, 'packet_loss': 9.0, 'bandwidth_mbps': 40, 'conn_type': 'wifi'}},

    # --- Group 6: Low Bandwidth Only ---
    {'desc': "Data Saver Mode (WiFi)", 'params': {'latency_ms': 80, 'packet_loss': 0.2, 'bandwidth_mbps': 2, 'conn_type': 'wifi'}},
    {'desc': "End of Data Plan (Cell)", 'params': {'latency_ms': 120, 'packet_loss': 1.0, 'bandwidth_mbps': 1, 'conn_type': 'cellular'}},

    # --- Group 7: Testing Bin Boundaries ---
    {'desc': "Boundary: Latency", 'params': {'latency_ms': 150, 'packet_loss': 0.5, 'bandwidth_mbps': 80, 'conn_type': 'wifi'}},
    {'desc': "Boundary: Bandwidth", 'params': {'latency_ms': 70, 'packet_loss': 0.5, 'bandwidth_mbps': 25, 'conn_type': 'wifi'}},
    {'desc': "Boundary: Packet Loss", 'params': {'latency_ms': 90, 'packet_loss': 3.0, 'bandwidth_mbps': 50, 'conn_type': 'cellular'}},
    
    # --- Group 8: Extra Scenarios for Coverage ---
    {'desc': "Excellent Lat, OK BW", 'params': {'latency_ms': 40, 'packet_loss': 0.1, 'bandwidth_mbps': 30, 'conn_type': 'wifi'}},
    {'desc': "OK Lat, Excellent BW", 'params': {'latency_ms': 95, 'packet_loss': 0.1, 'bandwidth_mbps': 200, 'conn_type': 'wifi'}},
    {'desc': "Great Cellular", 'params': {'latency_ms': 65, 'packet_loss': 0.2, 'bandwidth_mbps': 100, 'conn_type': 'cellular'}},
    {'desc': "Slightly Unstable WiFi", 'params': {'latency_ms': 85, 'packet_loss': 2.0, 'bandwidth_mbps': 70, 'conn_type': 'wifi'}},
    {'desc': "Video Call on 4G", 'params': {'latency_ms': 100, 'packet_loss': 0.8, 'bandwidth_mbps': 20, 'conn_type': 'cellular'}},
    {'desc': "Train WiFi", 'params': {'latency_ms': 450, 'packet_loss': 6.0, 'bandwidth_mbps': 8, 'conn_type': 'wifi'}},
    {'desc': "Downloading on Fast Cell", 'params': {'latency_ms': 80, 'packet_loss': 0.1, 'bandwidth_mbps': 250, 'conn_type': 'cellular'}},
    {'desc': "Barely Usable Signal", 'params': {'latency_ms': 900, 'packet_loss': 2.0, 'bandwidth_mbps': 2, 'conn_type': 'cellular'}},
    {'desc': "Extreme Latency", 'params': {'latency_ms': 5000, 'packet_loss': 1.0, 'bandwidth_mbps': 20, 'conn_type': 'wifi'}},
    {'desc': "Extreme Packet Loss", 'params': {'latency_ms': 100, 'packet_loss': 18.0, 'bandwidth_mbps': 20, 'conn_type': 'wifi'}},
]

# --- Print a header for the results table ---
print(f"{'Description':<25} | {'Recommendation':<15} | {'P2P Q-Value':<12} | {'Server Q-Value':<15} | {'Reason':<28} | {'State'}")
print("-" * 125)

# --- Loop through scenarios and print results ---
for scenario in test_scenarios:
    params = scenario['params']
    result = get_robust_recommendation(
        params['latency_ms'],
        params['packet_loss'],
        params['bandwidth_mbps'],
        params['conn_type'],
        q_table
    )
    
    details = result['details']
    rec = result['recommendation'].upper()
    p2p_q = f"{details.get('p2p_q_value', 'N/A'):.4f}" if details.get('p2p_q_value') is not None else 'N/A'
    server_q = f"{details.get('server_q_value', 'N/A'):.4f}" if details.get('server_q_value') is not None else 'N/A'
    
    print(f"{scenario['desc']:<25} | {rec:<15} | {p2p_q:<12} | {server_q:<15} | {details['decision']:<28} | {details['state']}")

Description               | Recommendation  | P2P Q-Value  | Server Q-Value  | Reason                       | State
-----------------------------------------------------------------------------------------------------------------------------
Perfect WiFi              | P2P             | 0.9956       | 0.7500          | model_exploitation           | lat_0_loss_0_bw_3_wifi
Excellent 5G Cellular     | P2P             | 0.7932       | 0.7500          | model_tie-break_preference   | lat_0_loss_0_bw_3_cellular
Fast Cable WiFi           | P2P             | 0.9956       | 0.7500          | model_exploitation           | lat_0_loss_0_bw_3_wifi
Average Home WiFi         | P2P             | 0.7075       | 0.7500          | model_tie-break_preference   | lat_0_loss_0_bw_2_wifi
Good 4G Cellular          | SERVER          | 0.5203       | 0.7500          | model_exploitation           | lat_1_loss_1_bw_2_cellular
Coffee Shop WiFi          | SERVER          | 0.5000       | 0.7500          | model_

In [94]:
# --- Step 6: Export the Trained Model for Production ---

# The "model" consists of the Q-table and the binning information.
# We'll save them together in a single JSON file.

# Get the bin edges as lists
latency_bin_edges = latency_bins.tolist()
loss_bin_edges = loss_bins.tolist()
bw_bin_edges = bw_bins.tolist()

# Convert the Q-table from a DataFrame to a nested dictionary
q_table_dict = q_table.to_dict(orient='index')

# Combine everything into one dictionary
production_model = {
    "model_name": "P2P-Server-RL-Agent",
    "version": 1.0,
    "creation_date": pd.Timestamp.now().isoformat(),
    "bin_edges": {
        "latency": latency_bin_edges,
        "loss": loss_bin_edges,
        "bandwidth": bw_bin_edges
    },
    "q_table": q_table_dict
}

import json

model_file_path = 'q_table_model.json'
with open(model_file_path, 'w') as f:
    json.dump(production_model, f, indent=4)

print(f"\n--- Model successfully exported to '{model_file_path}' ---")
print("This file contains everything your app needs to make decisions.")


--- Model successfully exported to 'q_table_model.json' ---
This file contains everything your app needs to make decisions.
